In [ ]:
left  = None
right = None
pedr_list = None
config1 = None
config2 = None
output_path = None

# Stereo Config file contents:

In [ ]:
if config2 == None:
    config2 = config1

In [ ]:
!cat {config1}

In [ ]:
!cat {config2}

# Setup Steps

In [ ]:
from IPython.display import Image
from asap_stereo import asap

In [ ]:
default_output_dir = '~/auto_asap/ctx/'
left, right = asap.CTX().get_ctx_order(left, right)
if output_path == None:
    output_path = default_output_dir + f'a_{left}_{right}'

In [ ]:
!mkdir -p {output_path}

In [ ]:
%cd {output_path}

# Download images (step 1)

In [ ]:
!asap ctx-one {left} {right} 2>&1 | tee -i -a ./1_download.log ./full_log.log

# First Step of CTX processing lev1eo (Step 2)

In [ ]:
!ctxedr2lev1eo.sh ./pair.lis 2>&1 | tee -i -a ./2_preprocess.log ./full_log.log

# Calculate BA and low-res DEM (Step 3)

In [ ]:
!asp_ctx_lev1eo2dem.sh -s {config1} -p ./pair.lis 2>&1 | tee -i -a ./3_ba_and_first_dem.log ./full_log.log

## Good Pixel Map 

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_ba/{both}_ba-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

# Calculate Better DEM using prior (Step 4)

In [ ]:
!asp_ctx_step2_map2dem.sh -s {config2} -p ./pair.lis 2>&1 | tee -i -a ./4_better_dem.log ./full_log.log

## Hillshade of low res DEM

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_ba/dem/{both}_ba_100_fill50-hillshade.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

## Good Pixel Preview

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_map_ba/{both}_map_ba-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

## Hillshade of higher res DEM

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_map_ba/dem/{both}_map_ba-hillshade.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

# Get PEDR Shots for PC alignment (Step 5)

In [ ]:
!pedr_bin4pc_align.sh {pedr_list} 2>&1 | tee -i -a ./5_pedr_query.log ./full_log.log

In [ ]:
!cat ./{left}_{right}/PEDR2TAB.PRM

In [ ]:
!cat ./{left}_{right}/{left}_{right}_pedr4align.csv | wc -l